<a href="https://colab.research.google.com/github/thomaschen20/satelite-damage-detection/blob/master/fine_tune_model_vector_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from damage_detection_functions import *
import matplotlib.pyplot as plt
import time
import os
import copy
import json
import random
from PIL import Image
import pprint
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.6.0+cu101
Torchvision Version:  0.7.0+cu101


In [4]:

model_name = "resnet"

# Number of classes in the dataset
num_classes = 4

# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train for
num_epochs = 100

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

input_type = 'pre_and_post' #pre_and_post, post_only, or pre_post_damage_type

In [5]:
with open(os.path.join('/content/drive/My Drive/Research Project', 'dataset_google_drive.json')) as f:
   root_dir = json.load(f)
for image_filename in root_dir:
    print(image_filename)
    print(root_dir[image_filename]['damage_type'])
    break
#print(len(sorted(list(root_dir.keys()))))
#print(root_dir[image])

/content/drive/My Drive/Research Project/train/images/guatemala-volcano_00000000_post_disaster.png
volcano


In [6]:
def train_model(model, dataloaders, criterion, optimizer, model_type, num_epochs=25):
    assert model_type in ['post_only', 'pre_and_post', 'pre_post_damage_type']

    since = time.time()

    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []



    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            if model_type == 'post_only':
              # Iterate over data.
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)
                  # zero the parameter gradients
                  optimizer.zero_grad()
                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      
                      # Get model outputs and calculate loss
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      _, preds = torch.max(outputs, 1)
                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()
            elif model_type == 'pre_and_post':
              # Iterate over data.
              for pre_inputs, post_inputs, labels in dataloaders[phase]:
                  processed_inputs = torch.cat([post_inputs, pre_inputs], dim=0)
                  processed_inputs = processed_inputs.to(device)
                  labels = labels.to(device)
                  # zero the parameter gradients
                  optimizer.zero_grad()
                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      # Get model outputs and calculate loss
                      outputs = model(processed_inputs)
                      loss = criterion(outputs, labels)
                      _, preds = torch.max(outputs, 1)
                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()
            elif model_type == 'pre_post_damage_type':
              # Iterate over data.
              for pre_inputs, post_inputs, labels, damage_type_input in dataloaders[phase]:
                  one_hot = torch.zeros(6)
                  one_hot[damage_type] = 1
                  processed_inputs = torch.cat([post_inputs, pre_inputs, one_hot], dim=0)
                  processed_inputs = processed_inputs.to(device)
                  labels = labels.to(device)
                  # zero the parameter gradients
                  optimizer.zero_grad()
                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      # Get model outputs and calculate loss
                      outputs = model(processed_inputs)
                      loss = criterion(outputs, labels)
                      _, preds = torch.max(outputs, 1)
                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        if phase == 'val':
            val_acc_history.append(epoch_acc)
            val_loss_history.append(epoch_loss)
        else:
            train_acc_history.append(epoch_acc)
            train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_acc_history, val_acc_history, train_loss_history, val_loss_history

In [ ]:
#mean = np.array([0.485, 0.456, 0.406])
#std = np.array([0.229, 0.224, 0.225])

In [ ]:
#dataloaders = dataloaders_dict
#for inputs, labels in dataloaders['train']:
#    print(inputs.shape)
#    for i in range(8):
#        image = np.array(inputs[i])
#        image = np.transpose(image, (1, 2, 0))
#        image *= std
#        image += mean
#        image *= 255
#        image = image.astype("uint8")
#        plt.imshow(image)
#        plt.show()
#        print(labels[i])
#    break

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        
        num_ftrs = model_ft.fc.in_features
        
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [ ]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
model_ft = nn.Sequential(*(list(model_ft.children())[:-1]))
# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Co

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
test_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
model_ft = model_ft.to(device)
def output_from_cnn(model_ft, crop):
    image_tensor = test_transforms(crop).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = image_tensor
    input = input.to(device)
    
    output = model_ft(input)
    return output

In [ ]:
!pip install --upgrade Pillow

     |████████████████████████████████| 2.2MB 3.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [ ]:
from tqdm import tqdm
for image_filename in tqdm(root_dir):
    image = cv2.imread(image_filename)[:,:,::-1].astype("uint8")
    if 'post_disaster' in image_filename:
        path = '/content/drive/My Drive/Research Project/tensors/post'
    else:
        path = '/content/drive/My Drive/Research Project/tensors/pre'
    building_index = 0
    for building in root_dir[image_filename]['building_list']:
        bbox = building['bbox']
        x, y, width, height = bbox
        if ('pre_disaster' in image_filename) or (building['damage_level'] != 'un-classified'):
            crop = image[int(y):int(y+height), int(x):int(x+width), :]
            try:
              vector = output_from_cnn(model_ft, Image.fromarray(crop))
              vector = vector.cpu().detach().numpy()
              np.save(os.path.join(path, image_filename[54:-4] + '_tensor_' + str(building_index).zfill(5) + '.npy'), vector)
            except:
              print("didn't work")
              print(image_filename + ": building #" + str(building_index))
        building_index += 1
        

 45%|████▍     | 2507/5598 [2:19:44<7:35:16,  8.84s/it]

didn't work
/content/drive/My Drive/Research Project/train/images/mexico-earthquake_00000023_pre_disaster.png: building #0


 45%|████▌     | 2535/5598 [2:23:15<8:06:30,  9.53s/it]

didn't work
/content/drive/My Drive/Research Project/train/images/mexico-earthquake_00000042_pre_disaster.png: building #0


100%|██████████| 5598/5598 [5:42:59<00:00,  3.68s/it]


In [ ]:
# import random
# model_post = nn.Linear(in_features=512, out_features=4, bias=True)
# model_post_and_pre = nn.Linear(in_features=512*2, out_features=4, bias=True)
# model_post_and_pre_and_damage_type = nn.Linear(in_features=512*2+7, out_features=4, bias=True)

# post = torch.rand(512)
# pre = torch.rand(512)
# damage_type = random.randint(0, 6)
# print("post shape:", post.shape)
# print("pre shape:", pre.shape)
# print("damage_type:", damage_type)
# val = get_post_output(post)
# print(val)
# val = get_post_and_pre_output(post, pre)
# print(val)
# val = get_post_and_pre_output_and_damage_type_output(post, pre, damage_type)
# print(val)

In [ ]:

# def train_model(models, dataloaders, type="post_only"):
#     for input, label in dataloaders["train"][type]:
#         if type == "post_only":
#             processed_input = input
#         elif type == "postandpre":
#             processed_input = concat()
#         elif type == 
#         output = models[type](processed_input)

In [9]:
class DamageDetectionDataset(Dataset):
    
    def __init__(self, 
                 root_dir : dict, 
                 input_type = 'post_only', 
                 equal_distribution = True, 
                 add_damage_type = False, 
                 minimum_area = 2000, 
                 split = 'train'):
        assert split in ["train", "val"]
        assert input_type in ["post_only", "pre_and_post"]
        assert add_damage_type in [True, False]
        assert equal_distribution in [True, False]
        self.root_dir = root_dir
        self.input_type = input_type
        self.add_damage_type = add_damage_type
        self.minimum_area = minimum_area
        self.split = split
        self.equal_distribution = equal_distribution
        self.images = [] #list of tuples
        
        if self.input_type == 'post_only':
            if self.split == 'train':
                post_images, _ = split_dataset_post_disaster(self.root_dir, percent_train= 0.8)
            else:
                _ , post_images = split_dataset_post_disaster(self.root_dir, percent_train = 0.8)

            for image_filename in post_images:
                building_index = 0
                for building in post_images[image_filename]['building_list']:
                    bbox = building['bbox']
                    if bbox[2] * bbox[3] >= self.minimum_area and building['damage_level'] != 'un-classified': #if it's not blurry
                        self.images.append(((image_filename, bbox, building['damage_level'], building_index), 0, post_images[image_filename]['damage_type'])) #0 is placeholder
                    building_index += 1
                        
        elif self.input_type == 'pre_and_post':
            if self.split == 'train':
                all_images, _ = split_dataset_pre_and_post_disaster(root_dir, percent_train=0.8)
            else:
                _ , all_images = split_dataset_pre_and_post_disaster(root_dir, percent_train=0.8)
            temp_image_list = sorted(list(all_images.keys()))
            for i in range(1, len(temp_image_list), 2):
                post_image_filename = temp_image_list[i-1]
                pre_image_filename = temp_image_list[i]
                building_index = 0
                for post_building in all_images[post_image_filename]['building_list']:
                    post_bbox = post_building['bbox']
                    if post_bbox[2] * post_bbox[3] >= self.minimum_area and post_building['damage_level'] != 'un-classified':
                        pre_building = all_images[pre_image_filename]['building_list'][building_index] #corresponding pre-disaster building
                        pre_bbox = pre_building['bbox']
                        post_damage_level = post_building['damage_level']
                        post_tuple = (post_image_filename, post_bbox, post_damage_level, building_index)
                        pre_tuple = (pre_image_filename, pre_bbox, 0, building_index)
                        self.images.append((post_tuple, pre_tuple, all_images[post_image_filename]['damage_type']))
                    building_index += 1
            
            
                    
        if self.equal_distribution:
            random.shuffle(self.images)
            if split == 'train': #smaller dataset of equal distribution
                per_category = 400
            else:
                per_category = 100
            dest = 0
            major = 0
            minor = 0
            none = 0 #starting with 0 buildings from each category
            temp_images = [] #temporary storage
            for buildings in self.images: #building is a tuple - if pre_and_post, then it's tuple of length 2 (pair of buildings)
                dl = buildings[0][2] #building[0] is always the post image no matter what
                if dl == 'un-classified':
                    continue
                elif dl == 'destroyed' and dest < per_category:
                    dest += 1
                    temp_images.append(buildings)
                elif dl == 'major-damage' and major < per_category:
                    major += 1
                    temp_images.append(buildings)
                elif dl == 'minor-damage' and minor < per_category:
                    minor += 1
                    temp_images.append(buildings)
                elif dl == 'no-damage' and none < per_category:
                    none += 1
                    temp_images.append(buildings)
                if dest == per_category and major == per_category and minor == per_category and none == per_category:
                    break
            self.images = temp_images #self.images now has an equal distribution of each category of building damage
            
            
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        if self.input_type == "post_only":
            (image_filename, bbox, damage_level, building_index), _, damage_type = self.images[idx]
            path = '/content/drive/My Drive/Research Project/tensors/post'
            tensor_filename = os.path.join(path, image_filename[54:-4] + '_tensor_' + building_index.zfill(5) + '.npy')
            damage_mapping = {'no-damage' : 0, 'minor-damage' : 1, 'major-damage' : 2, 'destroyed' : 3}
            type_mapping = {'earthquake' : 0, 'fire' : 1, 'flooding' : 2, 'tsunami' : 3, 'volcano' : 4, 'wind' : 5}
            sample = {'tensor': np.load(tensor_filename), 
                      'damage_level' : damage_mapping[damage_level], 
                      'damage_type' : type_mapping[damage_type]}
            if add_damage_type:
              return sample['tensor'], sample['damage_level'], sample['damage_type']
            else:
              return sample['tensor'], sample['damage_level']
        
        elif self.input_type == 'pre_and_post':
            (post_image_filename, post_bbox, damage_level, building_index), (pre_image_filename, pre_bbox, _, _), damage_type = self.images[idx]
            path = '/content/drive/My Drive/Research Project/tensors/pre'
            pre_tensor_filename = os.path.join(path, pre_image_filename[54:-4] + '_tensor_' + str(building_index).zfill(5) + '.npy')
            path = '/content/drive/My Drive/Research Project/tensors/post'
            post_tensor_filename = os.path.join(path, post_image_filename[54:-4] + '_tensor_' + str(building_index).zfill(5) + '.npy')
            damage_mapping = {'no-damage' : 0, 'minor-damage' : 1, 'major-damage' : 2, 'destroyed' : 3}
            type_mapping = {'earthquake' : 0, 'fire' : 1, 'flooding' : 2, 'tsunami' : 3, 'volcano' : 4, 'wind' : 5}
            sample = {'pre_tensor' : np.load(pre_tensor_filename), 
                      'post_tensor' : np.load(post_tensor_filename), 
                      'damage_level' : damage_mapping[damage_level], 
                      'damage_type' : type_mapping[damage_type]}
            if add_damage_type:
              return sample['pre_tensor'], sample['post_tensor'], sample['damage_level'], sample['damage_type']
            else:
              return sample['pre_tensor'], sample['post_tensor'], sample['damage_level']

In [10]:


print("Initializing Datasets and Dataloaders...")



# Create training and validation datasets
image_datasets = {x: DamageDetectionDataset(root_dir = root_dir, 
                                            input_type = input_type, 
                                            minimum_area = 2000, 
                                            split = x) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [11]:
if input_type == 'post_only':
  model_ft = nn.Linear(in_features=512, out_features=4, bias=True)
elif input_type == 'pre_and_post':
  model_ft = nn.Linear(in_features=512*2, out_features=4, bias=True)
elif input_type == 'pre_post_damage_type':
  model_ft = nn.Linear(in_features=512*2+7, out_features=4, bias=True)

In [12]:
# Send the model to GPU
model_ft = model_ft.to(device)
learning_rate = 0.001
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


Params to learn:
	 weight
	 bias


In [13]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [14]:
# Train and evaluate

# dataloaders = {
#     "train": {
#         "post_only": dataloader_post_only
#         "pre_and_post": dataloader_pre_and_post
#         "pre_post_damge_type": dataloader_pre_post_damage_type
#     }, 
#     "val": {
#         "post_only": post_model
#         "pre_and_post": pre_and_post_model
#         "sdffggg": thirdmodel
#     }
# }
# models = {
#     "post_only": post_model
#     "pre_and_post": pre_and_post_model
#     "sdffggg": thirdmodel
# }



model_ft, train_acc_hist, val_acc_hist, train_loss_hist, val_loss_hist = train_model(model_ft, 
                                                                                     dataloaders_dict, 
                                                                                     criterion, 
                                                                                     optimizer_ft, 
                                                                                     num_epochs=num_epochs, 
                                                                                     model_type = input_type)

Epoch 0/99
----------


OSError: ignored

In [16]:
np.load('/content/drive/My Drive/Research Project/tensors/pre/hurricane-michael_00000220_pre_disaster_tensor_00028.npy')

OSError: ignored

In [ ]:
epochs = range(num_epochs)
plt.plot(epochs, train_acc_hist, color = 'blue', label = 'Train')
plt.plot(epochs, val_acc_hist, color = 'orange', label = 'Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("Accuracy")
plt.show()


epochs = range(num_epochs)
plt.plot(epochs, train_loss_hist, color = 'blue', label = 'Train')
plt.plot(epochs, val_loss_hist, color = 'orange', label = 'Validation')
plt.title('Cross Entropy Loss')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Loss')
plt.legend()
plt.savefig("Cross Entropy Loss")
plt.show()


In [ ]:
model_name = ''
torch.save(model_ft.state_dict(), os.path.join("C:/Users/thoma/OneDrive/Damage Detection", model_name))